#Predicting Flight Delays for US Airlines
##__w261 Final Project - Phase 3 - Team 3-3__

## I. Team Members

| Name | Email | Photo | Location|
| --- | --- | --- | ---|
| Daniel Florencio | dcflorencio@berkeley.edu | <img src="https://ca.slack-edge.com/T0WA5NWKG-U05H59CT9T3-7d2f162be084-512" alt="Daniel Florencio" width="300" height="300"> | Seattle, WA |
| Dong Jian | an.dong90@berkeley.edu | <img src="https://ca.slack-edge.com/T0WA5NWKG-U04BVDG8R0F-40d98c327922-512" alt="Dong Jian" width="300" height="300"> | San Francisco, CA |
| Jacqueline Yeung | jtyeung168@berkeley.edu | <img src="https://ca.slack-edge.com/T0WA5NWKG-U05KR42JS4C-5244b24e03ce-512" alt="Jacqueline Yeung" width="300" height="300"> | Union City, CA |
| Karina Kowarsch | karinakowarsch@berkeley.edu | <img src="https://ca.slack-edge.com/T0WA5NWKG-U049P4RUB6Y-427be2112e65-512" alt="Karina Kowarsch" width="300" height="300"> | Riverside, CA |

## II. Phase Leader Plan

| Week | Week/Start| Phase | Leader | Backup/Co-Leader |
| --- | --- | --- | --- | --- |
| 1 | July 8 | 1 | Daniel | Dong |
| 2 | July 15 | 2 | Dong | Jacqueline |
| 3 | July 22 | 2 | Jacqueline | Karina |
| **4** | **July 29** | **3** | **Karina** | **Daniel**|
| **5** | **Aug 5** | **3** | **Daniel** | **Dong** |

## III. Credit Assigment Plan
| Phase | Task | Contributors | Contributions (hours) |
| --- | --- | --- | --- |
| 3 | Planning the pipeline | Jacqueline, Dong, Daniel, Karina | 3 |
| 3 | Data transformation: custom join | Daniel | 4 |
| 3 | Data prep: pruning fields | Dong | 0.5 |
| 3 | Data prep: handling missing data | Jacqueline | 3 |
| 3 | EDA | Dong, Daniel, Jacqueline, Karina | 3 |
| 3 | Feature Engineering: feature selection | Karina, Jacqueline | 2 |
| 3 | Feature Engineering: feature generation | Dong, Daniel, Karina, Jacqueline | 4 |
| 3 | Generating train/test and CV splits | Jacqueline, Dong | 2 |
| 3 | Rebalancing train CV splits | Dong | 0.5 |
| 3 | Baseline Model: Logistic Regression | Dong, Karina | 1 |
| 3 | Model Experiment: MLP Model | Dong | 3 |
| 3 | Model Experiment: XGBoost Model | Karina | 3 |
| 3 | Model Experiment: Random Forest Model | Jacqueline | 3 |
| 3 | Model Experiment: Ensembled Model | Dong | 5 |
| 3 | Working on presentation deliverable for Phase 3 | Daniel, Dong, Jacqueline, Karina | 3 |
| 3 | Writing Project Abstract | Daniel | 0.5 |
| 3 | Writing Data and Feature Engineering | Karina, Daniel, Jacqueline | 3 |
| 3 | Writing Neural Network (MLP) Description | Dong | 1 |
| 3 | Writing Project Modeling Pipelines | Jacqueline, Karina | 3 |
| 3 | Writing Results | Dong | 2 |
| 3 | Writing Project Conclusion | Dong | 0.5 |

## IV. Project Abstract
The Department of Transportation tasked us with predicting U.S. flight delays at least two hours in advance. We integrated flight, weather, and station data into the OTPW dataset, using publicly available data from the National Oceanic and Atmospheric Administration and the U.S. Department of Transportation. The dataset spans from 2015 to 2019, covering over 30 million flights and containing 200+ weather data columns. We utilized this data to develop models, starting with Logistic Regression and advancing to Decision Trees, XGBoost, and Neural Networks, using F-beta (beta = 2) as the key metric to balance precision and recall, with a focus on prioritizing recall.

Our baseline was a Logistic Regression model, chosen for its simplicity and effectiveness in binary classification tasks. It utilized features such as day of the month, day of the week, types of origin and destination, delay of the previous flight, and binary variables like lighting, precipitation, and icing. This baseline achieved an F-beta score (beta = 2) of 0.52.

Among the models developed, the Random Forest model emerged as the best performer, achieving a test F-beta score of 0.57. This improvement was driven by enhanced feature engineering, including the incorporation of historical flight delays, weather conditions, and school break periods. 

## V. Data and Feature Engineering


### Data Joining Pipeline
![](https://raw.githubusercontent.com/dojian/Airline_delay/main/data_join.JPG)


#### Introduction
This section details the data joining pipeline used to integrate four key datasets: 
- airport codes
- stations
- flights
- weather data.

#### Data Sources and Initial Filtering

**Airport Codes and Stations**
- Airport Codes Dataset: Initially, the dataset contained 57,421 rows and 12 columns. We filtered this to include only airports in the United States, resulting in a dataset with 2,031 rows and 12 columns.
- Stations Dataset: The original dataset comprised 2,237 rows and 12 columns.

**Joining Airports and Stations**
- Cross Join: A cross join was performed between the airport codes and stations datasets, both of which contained latitude and longitude information.
- Distance Calculation: The Haversine formula was used to calculate the distances between stations and airports.
- Filtering: Stations within a 50 km radius of an airport were retained, resulting in a smaller dataset.
- Ranking: The stations were ranked by distance for each IATA code, and the closest station for each airport was selected.
- Final Dataset: The resulting dataset, which includes IATA codes, contains 1,887 rows and 20 columns. This dataset will be crucial for subsequent joins.

#### Merging Flights Data

**Joining Airports/Stations with Flights**
- IATA Code Matching: The Airports/Stations table was joined with the flights table using the IATA codes and the `origin` column.
- Generated Table: The resulting table had 73,764,819 rows and 129 columns, integrating information from both flights and airport/station datasets.

#### Integrating Weather Data

**Preparing Data for the Weather Join**
- **Datetime Columns**: New columns were generated in the flights table:
  - `datetime_sched_crs`: Created using `CRS_DEP_TIME` and `FL_DATE`.
  - `datetime_lower_bound`: Set as 2 hours before the scheduled departure time.
  - `datetime_upper_bound`: Set as 4 hours before the scheduled departure time.

**Weather Data Join**
- Join Logic: The join was performed using the station ID and the datetime feature, matching records within the 2-hour window between the lower and upper bounds.
- Resulting Data: Each flight record was matched with approximately 5 different weather records, creating valuable "lag" features that can be used in future models.

#### Error Handling and Challenges

**Checkpointing**
- Checkpointing Process: Throughout the process, we checkpointed our tables up to the Airports/Stations/Flights join.
- Final Data Issue: Unfortunately, the final data checkpointing process did not complete, as the write operation would take many hours.

**Alternative Approaches**
- Window Reduction Attempt: We attempted to reduce the window to just 20 minutes (window between 2h and 2h20 prior), but this approach also failed to complete the checkpointing process.

#### Data Joining Conclusion
Despite challenges with the final data checkpointing, the steps taken to join airport, station, flight, and weather data have laid a strong foundation for subsequent analysis. Moving forward, efforts should focus on optimizing the data writing process or exploring alternative methods for efficiently handling large datasets in similar projects.


### Feature Families Explored
In the provided combined 5-year data, we explored the following feature families, leveraging existing raw features and/or generating new ones from each family for use in our modeling experiments.

#### Table 1: Raw and New Feature Families
The following table describes raw features and their corresponding newly-generated features, categorized by family. \
![](https://raw.githubusercontent.com/dojian/Airline_delay/main/Feature%20Mapping%202.png)

#### Table 2: Feature Families Descriptions and Counts
The following table enumerates the newly-generated features, categorized by family, along with their descriptions and counts.
| Family               | Features                                           | Description                                                                                        |Count|
|----------------------|---------------------------------------------------|---------------------------------------------------------------------------------------------------|---|
| Time-Based           | DEP_DELAY_Lag_1, is_prev_flight_delayed, is_prev_flight_delayed_15 | Features generated from the previous flight by `TAIL_NUM` and `sched_depart_date_time_UTC`. These features rerpresent the departure delay of the previous flight, wehther the previous flight was delayed, and whether the previous flight was delayed by >= 15 minutes, respectively.         |3|
| Graph-Based          | pagerank                                          | The origin airport's PageRank values.                                                                |1|
| Event-Based (binary)        | school_break                                     | Indicates whether or not there is a school break month.                                           |1|
| Weather-Based (binary)       | lighting, icing, precipitation                    | Based on temperature and relative humidity.                                                       |3|
| Airport Characteristics | origtype_desttype_vec                           | A concatenated feature based on the origin and destination airport types. i.e. small-small, large-medium, etc.,                         |1|
| Time Segmentation| dep_seg_vec                                       |The scheduled departure time segmentation of a given day. i.e.morning, evening, etc.,                                |1|




#### Figure 1: Feature Families vs. DEP_DELAY (in minutes)
The following bar charts illustrate a few of the features in the above families in relation to `DEP_DELAY`, the flight's departure delay in minutes.
![](https://raw.githubusercontent.com/dojian/Airline_delay/main/binary%20features%20EDA.png)

Figure 1 shows that among the weather-based features, precipitation and icing are positively correlated with departure delays, meaning that as the intensity of precipitation or icing increases, the likelihood or magnitude of departure delays also tends to increase. However, the lighting feature does not exhibit a similar positive correlation. Additionally, the event feature 'school break' is also positively correlated with departure delays.

#### Figure 2: School Break Feature (winter, spring, summer break)
![](https://raw.githubusercontent.com/dojian/Airline_delay/main/school_break.JPG) [](url)

As we explored the data, we observed that flight delays were consistently more significant during specific times of the year across all years analyzed. This pattern suggested a correlation with school break seasons. To capture this effect, we identified the winter, spring, and summer break periods and created a new binary feature named `school_break` to indicate whether a flight occurred during one of these school break periods.

#### Figure 3: Pagerank Feature
![](https://raw.githubusercontent.com/dojian/Airline_delay/main/top_20_airports_pagerank_count_prop_del_15.png) \
The left-most barchart describes the top 20 origin airports by pagerank for the 5-year data. The pagerank is based on outgoing flights. From the plot we can see that certain airports serve as hubs in the U.S., with the Hartsfield-Jackson Atlanta International Airport serving as the largest hub, followed by the Chicago O'Hare International Airport. This seems to correlated closely with the top 20 airports by count of delayed flights (>= 15 minutes), illustrated in the bar chart in the center. However, if we compare this with the top 20 airports by _proportion_ of delayed flights (>= 15 minutes), illustrated in the right-most bar chart, we see that the pagerank may not be as indicative of delay as it seems. Nonetheless, we include this feature in our analysis. 

### Experiments and the Value of Each Feature/Family
We performed feature selection techniques for the following models we experimented with. For some of the experiments we employed the following 2 feature selection techniques:
1. Categorical features + categorical label
      - Chi-squared test: statistical test to test whether features are dependent on the response (DEP_DEL15)
2. Continuous features + categorical label
      - ANOVA F-test: statistical test to test how well features discriminate between the response classes (DEP_DEL15)

#### XGBoost
The model used numerical features such as school break, lighting, precipitation, icing, hourly visibility, and dep_delay_lag, along with all one-hot encoded features. It applied the Chi-squared method to select categorical features and numerical features (when converted to intervals) by evaluating how well they separate the classes "delay15" and "not delay15," with features having higher Chi-squared values being more significant. The selected features are QUARTER, DAY_OF_MONTH, DAY_OF_WEEK, OP_UNIQUE_CARRIER, ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,DEST_CITY_MARKET_ID,DEST.  The required numerical features are lighting, precipitation, icing, HourlyVisibility, HourlyWindSpeed, Distance, school_break, and DEP_DELAY_lag_1, with two one-hot encoded features: origtype-desttype_vec and dep_seg_vec.

#### Random Forest / Ensembled Model
The model considered as categorical features, attributes such as quarter, year/month/day of month/day of week, tail/flight number, origin/destination airport code/state, departure/arrival time block, distance group, origin/destination airport type, lighting, precipitation, icing, departure time segment, previous flight is delayed, and school break. It considered as numerical features, attributes such as the flight's elapsed time, flight distance, origin airport latititude/longitude, elevation, previous departure delay, pagerank, and the numerical weather features. We used the Chi-squared method to select the top categorical features and the ANOVA F-test method to select the top numerical features. From this process, top features included CRS_ELAPSED_TIME, DAY_OF_WEEK, OP_CARRIER_FL_NUM, DAY_OF_MONTH, QUARTER, dest_airport_lat, origin_airport_lat, origin_airport_lon, ELEVATION, DISTANCE, TAIL_NUM, dest_airport_lon, and DEP_DELAY_lag_1.

#### Other Models
For other models we experimented with, we hand-picked features to use. These included the scheduled depart time, day of month, hourly dry bulb temperature/relative humidity, tail number, and origin/destination airport type.

## VI. Neural Network (MLP)
We performed grid search(neurons for 1 layer[10,30,50], neurons for 2 layers[5,4;20,10]) and here are three Multilayer Perceptron(MLP) models helping us understanding the parameters with different architectures:

- Model 1: 2 layers with 5 and 4 neurons respectively. Runtime: 39mins
- Model 2: 1 layer with 10 neurons. Runtime: 37mins
- Model 3: 1 layer with 50 neurons. Runtime: 3hrs

All of these three models have maxIter=100,and blockSize=128 as settings. And below are the performance on 5 folds(2015-2018) train and test(2019) datasets.

<img src="https://raw.githubusercontent.com/dojian/Airline_delay/main/output%20(2).png" width="500" height="500">     |

<img src="https://raw.githubusercontent.com/dojian/Airline_delay/main/output%20(3).png" width="450" height="450"> <img src="https://raw.githubusercontent.com/dojian/Airline_delay/main/output%20(4).png" width="450" height="450">

Model 1 (2 layers: 5, 4 neurons) showed strong training performance but struggled with slight overfitting, leading to a drop in test set performance. Model 2 (1 layer: 10 neurons), with its simpler architecture, balanced complexity and generalization more effectively, achieving the best test set results with a higher F-beta score and consistent recall. Model 3 (1 layer: 50 neurons) exhibited the poorest generalization, likely due to over-parameterization, resulting in the lowest test set performance. These findings underscore the importance of balancing model complexity with generalization needs and highlight the value of simpler, well-tuned models in achieving robust performance across unseen data.

## VII. Leakage
### What is Leakage?
Data leakage occurs when a machine learning model learns from data it is not supposed to "see". In this case, we are tasked with predicting whether `DEP_DEL15 = 1` using data available 2 hours or prior to each flight's scheduled deperature time. This means that if any of our machine learning models are trained on data within this 2 hour window or after the flight's scheduled departure, the model is invalid and there is data leakage. Examples of features that contribute to data leakage are the current flight's during-flight and post-flight features--e.g. `ARR_DEL15`, `ARR_DELAY`, `ACTUAL_ELAPSED_TIME`. This also includes delay-related features--e.g. `DEP_DELAY`, `CARRIER_DELAY`, `WEATHER_DELAY`---if they are within the 2 hour window. For example, knowing that a flight arrived late can serve as an indicator that the flight departed late and defeats the purpose of our task. Moreover, if any of our models are trained on data from future flights, that is data leakage. 

### Evaluating our Pipeline For Leakage
When designing our pipeline, we accounted for data leakage by or tried to minimize it as much as possible. In the sub-sections below, we discuss how we addressed data leakage at certain stages. For a full description of the stages in our pipeline, refer to **Section VIII. Modeling Pipelines** below. 
#### Lag Feature Generation (Pipeline Stage 3)
To account for data leakage, we make certain assumptions when generating our lag features. We assume that a flight's DEP_DELAY_GROUP, DEP_DELAY, and DEP_DEL15 is only **determined** at the actual time of departure, and we only **know** the previous flight's DEP_DELAY_GROUP, DEP_DELAY, and DEP_DEL15 if its actual dep time <= curr two_hours_prior_depart_UTC. Otherwise, these will be **unknown** to us (masked).

We assume that the prev sched_depart_date_time_UTC is always **known** to us.

Given these assumptions, we generate our 3 lag features as follows:
1. is_prev_flight_delayed (1 if the previous flight is delayed; 0 if the previous flight is on time or early)
    - if prev actual dep time <= curr sched dep time 2 hrs prior, then value of prev DEP_DELAY > 0.0
    - else if prev sched dep time <= curr sched dep time 2 hrs prior, then 1 
    - null otherwise
2. is_prev_flight_delayed_15 (1 if the previous flight is delayed >= 15 mins; 0 otherwise)
    - if prev actual dep time <= curr sched dep time 2 hrs prior, then value of prev DEP_DEL15 != 0
    - else if curr sched dep time 2 hrs prior - prev sched dep time >= 15 mins, then 1
    - null otherwise
3. DEP_DELAY_lag_1
    - if prev actual dep time <= curr sched dep time 2 hrs prior, then value of prev DEP_DELAY
    - null otherwise
#### Impute Missing Values (Pipeline Stage 4)
When imputing missing values we either impute with values that don't depend on time, such as default values `0` or `""`, or if they are dependent on time, we only consider past data. For example, we impute numerical weather features with the average of the 2 previous values, windowed by the aircraft number and ordered by the flight's scheduled depart date time.
#### Train/test Split (Pipeline Stage 5)
We ensure that the train split comprises of 2015-2018 data and the test split comprises of 2019 data. With this split we ensure that the heldout set is chronologically later that the train set and will not result in data leakage. In this way, we are also ensuring we don't violate the cardinal sin of ML wherein our models look at the heldout dataset when training.
#### CV Split (Pipeline Stage 6)
Similarly, when generating the cross-validation splits, we employ the rolling window strategy, which is slightly different from the typical cross-validation strategy. This rolling window strategy is dependent on time and ensures each validation fold is chronologically later than each train fold. Additionally, each train fold is chronologically later than the previous. This ensures our models never look at future data when training.
#### Pagerank Generation (Pipeline Stage 7)
To generate our graph feature, the origin airport pageranks, we minimize data leakage by generating the pageranks after the CV split. The pagerank algorithm we use is time-agnostic; hence, we addressed data leakage as follows. We generate the pageranks on each train fold and then merge (join) these pageranks with the corresponding validation fold and the next train fold (or test set if it is the last fold). In this way, each train and validation fold will have pageranks generated from past data. There is some slight data leakage in that for the first fold (fold 0), we merge the pageranks generated by the train fold to itself. However, we considered this unavoidable as there was no other means to generate the pageranks for this train fold. 

## VIII. Modeling Pipelines

### Project Flow
The image below describes our end-to-end project flow pipeline. \
![](https://raw.githubusercontent.com/dojian/Airline_delay/main/Modeling%20Pipeline.png)
The stages are as follows.
- **Business Understanding**
  - Understanding the business problem we want to solve, which is developing a predictive system for flight delays, based on certain constraints, for the DoT to use to inform airports and airlines in their flight scheduling.
- **Data Understanding**
  - Understanding the sources of the data (DoT, NOAA) and what the data attributes and records mean semantically.
  - Extract, load, and transform for analysis.
- **EDA (Exploratory Data Analysis)**
  - Perform exploratory data analysis to understand data characteristics.
  - Iterate between EDA and data prepping.
- **Data Prep (Data Preparation)**
  - Data cleaning to ensure quality.
  - Basic feature engineering to prepare data for modeling.
- **Modeling**
  - Train-test split and rolling window cross-validation split.
  - Rebalance train folds and perform feature selection/engineering.
  - Initialize the modeling pipeline and train models.
  - Iterate between modeling and assessment.
- **Assessment**
  - Generate predictions from the trained models and evaluate performance.
- **Deployment**
  - Deploy the best evaluated model to production at scale, according to a deployment strategy.
- **Monitoring**
  - Continuously monitor the deployed model’s performance for regressions.
  - Feedback loop to business understanding for continuous improvement.

### Data Pipeline
The image below describes our data pipeline in closer detail. This pipeline includes data prepration and splitting steps. We process the combined 5-year data in several stages and checkpoint to our blob storage after each. Checkpoints are indicated by the star symbol. \
![](https://raw.githubusercontent.com/dojian/Airline_delay/main/DATASCI%20261_%20Data%20Pipeline_phase3.png)
1. **Stage 1**: initial cleanse
- In this stage we prune the 214 fields in the 5-year dataset to just 85, removing features we believe are not immediately relevant to our analysis. These removed features include `Daily` and `Monthly`-level weather data and those that are redundant, unrelated to our business problem, or in a format (e.g. local time) that we aren't using. 
- As part of this stage, we also impute cancelled flights with null `DEP_DEL15` with 1. By imputing with 1, we are considering such flights as delayed. 
- We also check for rows with null `ORIGIN` and `DEST` as we cannot use them for our analysis. In this case, there were none.
2. **Stage 2**: basic feature generation
- In this stage we generate the following 6 features: 
  - `origtype-desttype`: a concatenation of `origin_type` and `destt_type` features. These features describe what kind of aiport (e.g. large_airport) the origin and destination airports correspond to and may have an impact on the delay.
  - `lighting`, `icing`, and `precipitation`: these 3 features are binary features derived from the `Hourly` weather features and serve as summary features that may indicate delay. We consulted with the current literature to generate these features.
    - `lighting`: 1 if `HourlyDryBulbTemperature` > 25 and `HourlyRelativeHumidity` < 30; else 0.
    - `precipitation`: 1 if `HourlyDryBulbTemperature` < 40; else 0.
    - `icing`: 1 if `HourlyDryBulbTemperature` < 32 and `HourlyRelativeHumidity` > 80; else 0.
  - `dep_seg`: derived from `DEP_TIME_BLK` and classifies the time blocks by category: "midnight", "E_morning", "morning", "mid_day", "afternoon", "evening", and "late night".
  - `school_break`: binary feature to denote if the month of the flight falls on a school break (months of Dec-Jan, Mar, July-Aug).
3. **Stage 3**: lag feature generation
- In this stage we generate the following 3 features based on the previous flight based on the `TAIL_NUM`, the aircraft number, and ordered by `sched_depart_date_time_UTC`:
  - `DEP_DELAY_lag_1`: a lag feature representing the `DEP_DELAY` of the previous flight.
  - `is_prev_flight_delayed`: binary lag feature representing whether the previous flight was delayed.
  - `is_prev_flight_delayed_15`: same as `is_prev_flight_delayed` except with additional condition of delayed by 15 or more minutes.
- For an explanation of how we addressed data leakage, refer back to section **VII. Leakage**.
4. **Stage 4**: impute missing values
- In this stage, we first identified why fields had missing values.
- Then we impute missing values with different strategies:
  - drop columns with >60% of its values being null. We do so because such fields are mostly null and likely not able to give us much information. We don't drop pre-flight features falling into this category, such as `CARRIER_DELAY`, `WEATHER_DELAY`, etc., because it does not makes sense to do so.
  - flight features: impute with 0. We do so because other features such as `DEP_DELAY`, `DIVERTED`, or `CANCELLED` should serve as indicators for why these features are null. 
  - weather features: for numerical features, impute with a the rolling average of the past 2 values, windowed by the `TAIL_NUM` and ordered by the `sched_depart_date_time_UTC`, or 0 if this average cannot be computed. Additionally, several of these numerical features have non-numeric characters in their values. We remove these non-numeric characters before casting the values to numeric type and immputing. For categorical features, impute with an empty string `""`.
  - lag features: impute with 0. We assume the previous flight was on-time. 
  - for remaining features, we drop the rows with null values. We do so because <20% of values for these features are null and for some of them, like `TAIL_NUM`, we won't be able to do our analysis on missing data. This also included cases where `DEP_DEL15` is null but the flight is not cancelled. Such occurrences were few and did not make sense to keep.
5. **Stage 5**: train-test split
- In this stage we split the resulting data into train and test sets where the train set comprises of 2015 - 2018 data and the test set comprises of 2019 data.
6. **Stage 6**: rolling-window CV split
- In this stage, we generate the rolling window cross-validation splits on the train set, forming 5 folds, each containing the corresponding train and validation sets. We choose the rolling window strategy for this time-series data because it is more scalable than the exapanding window strategy. To determine the rolling window, we sort on `sched_depart_date_time_UTC`.
7. **Stage 7**: generate pagerank graph feature
- In this stage we generate a feature called `pagerank` to represent the pagerank of each airport in the graph representation of the flights. The image below describes our process for performing this feature generation.\
![](https://raw.githubusercontent.com/dojian/Airline_delay/main/DATASCI%20261_%20Pagerank%20Generation.png)
    - the procedure is as follows:
      1. fold 0
          - generate pageranks for train fold 0
          - merge train fold 0 pageranks to train fold 0 (i.e. itself)
          - merge train fold 0 pageranks to val fold 0
      2. fold 1
          - generate pageranks for train fold 1
          - merge train fold 0 pageranks to train fold 1
          - merge train fold 1 pageranks to val fold 1 \
          and so on until the last fold ...
      5. fold 4
          - generate pageranks for train fold 4
          - merge train fold 4 pageranks to val fold 4
          - merge train fold 4 pageranks to test (holdout) set
8. **Stage 8**: rebalance CV train splits
- In this stage, we handle data imbalance on the train sets in each of the folds via downsampling (randombly selecting records to drop). We choose downsampling as our method for scalability. In the end the number of delayed vs non-delayed flights in each train set is around the same.

After these base stages are complete, subsequent stages (not illustrated) involve additional feature selection and engineering before initializing the ML pipeline and are flexible in nature.

#### Row Counts
The following table summarizes the row counts before and after each stage in our pipeline. 
- The "Intial row count" and "Final row count" measure the total row counts before and after the stage. 
- The "Final train row count" measures the row counts of each train set (non-heldout set or each CV train) after the stage.
- The "Final validation row count" measures the row counts of each CV validation set after the stage.
- The "Final test row count" measures the row count of the heldout test set after the stage.

|Stage|Initial row count|Final train row count|Final validation row count| Final test row count|Final row count|Difference|Blob storage locations|
| --- | --- | --- | --- | --- | --- | --- | --- |
1: initial cleanse|31673119||||31673119|0|{team_blob_url}/5yr_step1_basic_cleanse|
2: basic feature generation|31673119||||31673119|0|{team_blob_url}/5yr_step2_basic_feature_generation|
3: lag feature generation|31673119||||31673119|0|{team_blob_url}/5yr_step3_lag_feature_generation|
4: impute missing values|31673119||||31596156|-76963|{team_blob_url}/5yr_step4_imputed|
5: train-test split|31596156|24220147||7376009|31596156|0|{team_blob_url}/5yr_step5_train_df, {team_blob_url}/5yr_step5_holdout_df|
6: rolling-window CV split|24220147|4613362|1153341|||||
|||4613362|1153341|||||
|||4613362|1153341|||||
|||4613362|1153341|||||
|||4613362|1153337||24220147|0|{team_blob_url}/cross_validation/5yr_step6_cv_train_fold_{fold}, {team_blob_url}/cross_validation/5yr_step6_cv_validation_fold_{fold}|
7: pagerank generation|24220147|4613362|1153341|||||
|||4613362|1153341|||||
|||4613362|1153341|||||
|||4613362|1153341|||||
|||4613362|1153337||24220147|0|{team_blob_url}/cross_validation/5yr_step7_cv_train_pagerank_fold_{fold}, {team_blob_url}/cross_validation/5yr_step7_cv_val_pagerank_fold_{fold}, {team_blob_url}/5yr_step7_holdout_pagerank_df|
8: rebalance CV train splits|24220147|1837399|1153341|||||
|||1699931|1153341|||||
|||1702745|1153341|||||
|||1730975|1153341|||||
|||1820129|1153337||9944516|-14275631|{team_blob_url}/cross_validation/ 5yr_step8_downsampled_train_fold_fold_{fold}|

### Loss Functions
#### Binary Cross-Entropy Loss
For our baseline logistic regression model, neural network models, and XGBoost model, we use the following loss function.
$$
\mathcal{L} = - \frac{1}{N} \sum_{i=1}^{N} \left[ y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i) \right] 
$$
where
- 𝑁 is the number of samples.
- \\(y_i\\) is the actual label for the 𝑖-th sample. It can be either 0 or 1.
- \\(\hat{y}_i\\) is the predicted probability of the positive class (label 1) for the i-th sample, given by the model. It ranges from 0 to 1.

#### Gini Impurity
For our random forest model, we use the following loss function.
$$
\ Gini = 1 - \sum_{i=1}^{n} p_i^2 
$$
where
- \\(p_i\\) is the probability of an element being classified as class \\(i\\) 

## IX. Results
We trained and evaluated the model using a 5-fold cross-validation approach on the 2015 to 2018 dataset. Then test on the 2019 dataset. The main metric we used was F-beta with beta = 2.0, focusing on recall. The beta was determined using the cost of errors ratio between False Negative(FN) and False Positive(FP).
### Ratio of FN to FP cost
$$
\text{Ratio of FN to FP cost} = \frac{\text{Cost of FN}}{\text{Cost of FP}} = \frac{300}{150} = 2.0
$$

Here we estimated the cost in each senario using the table below. In both cases, passenger will need change/rebook their flights, but the cost is less on airline disruptions and accommodations such as hotel to customers when false positive happens than false nagative.

### Cost of Errors

| Description/ $ per flight per person                              | Cost of a False Positive (FP) | Cost of a False Negative (FN) |
|-------------------------------------------------------------------|-------------------------------|-------------------------------|
|                                          | Predict delay when it is not      |Predict No-delay when it is
| Average passenger rebooking                                       | 100                           | 100                           |
| Accommodations(hotel, meals)                                                    | 0                             | 100                           |
| Airline operational cost (e.g., compensations, operational disruptions) | 50                            | 100                           |
| **Total**                                                         | **150**                       | **300**                       |


### Experiments and Results
| #  | Experiment | Hyperparameters | Features | Loss Function | Runtime (min) | Results on train fold 0 | Results on train fold 1 | Results on train fold 2 | Results on train fold 3 | Results on train fold 4 | Results on test set |
|---|----------|---------------|--------|---------------|---------------|-------------------------|-------------------------|-------------------------|-------------------------|-------------------------|-------------------|
| 1 | Baseline: Logistic Regression | maxIter: 200 | 8 | Binary cross-entropy | 16 | F-2: 0.48<br>Precision: 0.24<br>Recall: 0.65 | F-2: 0.48<br>Precision: 0.23<br>Recall: 0.67 | F-2: 0.57<br>Precision: 0.34<br>Recall: 0.69 | F-2: 0.50<br>Precision: 0.26<br>Recall: 0.66 | F-2: 0.51<br>Precision: 0.25<br>Recall: 0.57 | F-2: 0.52<br>Precision: 0.28<br>Recall: 0.67 |
| 2 | MLP (Multilayer Perceptron) | One hidden-layer(10 neutrons,maxIter=100,blockSize=128) | 8 | Binary cross-entropy | 37 | F-2: 0.43<br>Precision: 0.28<br>Recall: 0.50 | F-2: 0.46<br>Precision: 0.26<br>Recall: 0.57 | F-2: 0.54<br>Precision: 0.38<br>Recall: 0.60 | F-2: 0.54<br>Precision: 0.31<br>Recall: 0.67 | F-2: 0.54<br>Precision: 0.32<br>Recall: 0.65 | F-2: 0.51<br>Precision: 0.26<br>Recall: 0.66 |
| 3 | MLP (Multilayer Perceptron) | Two hidden-layers(5 and 4 neutrons,maxIter=100,blockSize=128) | 8 | Binary cross-entropy | 39 | F-2: 0.45<br>Precision: 0.29<br>Recall: 0.53 | F-2: 0.48<br>Precision: 0.27<br>Recall: 0.58 | F-2: 0.59<br>Precision: 0.38<br>Recall: 0.68 | F-2: 0.54<br>Precision: 0.35<br>Recall: 0.63 | F-2: 0.54<br>Precision: 0.35<br>Recall: 0.63 | F-2: 0.47<br>Precision:0.23<br>Recall:0.65 |
| 4 | XGBoost | num_workers = 5, max_depth = 10, eta = 0.1 | 20 | Binary cross-entropy | 60 | F-2: 0.43<br>precision:0.42<br>recall:0.43 | F-2:0.53<br>precision:0.32<br>recall:0.63 | F-2:0.62<br>precision:0.42<br>recall:0.71 | F-2:0.57<br>precision:0.36<br>recall:0.66 | F-2:0.56<br>precision:0.40<br>recall:0.63| F-2: 0.56<br>Precision:0.40<br>Recall:0.63 |
| 5 | RF (Random Forest) | maxDepth: 3, numTrees: 10, impurity: gini | 13 | Gini Impurity | 58 | F-2: 0.55<br>Precision: 0.27<br>Recall: 0.74 | F-2: 0.51<br>Precision: 0.25<br>Recall: 0.70 | F-2: 0.61<br>Precision: 0.34<br>Recall: 0.75 |  F-2: 0.56<br>Precision: 0.29<br>Recall: 0.72 | F-2: 0.57<br>Precision: 0.30<br>Recall: 0.72 | F-2: 0.57<br>Precision: 0.31<br>Recall: 0.72 |
| 6 | Ensembled model | XGboost(num_workers = 3, max_depth = 5, eta = 0.2), RF(maxDepth: 3, numTrees: 3, impurity: gini), Logistic regression(maxIter100) | 13 | Binary cross-entropy | 60 | F-2:0.41<br>precision:0.37<br>recall:0.42 | F-2:0.52<br>precision:0.27<br>recall:0.68 | F-2:0.57<br>precision:0.40<br>recall:0.65 | F-2:0.56<br>precision:0.31<br>recall:0.71 | F-2:0.52<br>precision:0.38<br>recall:0.58 | F-2:0.55<br>precision:0.37<br>recall:0.62 |

* computation resources for all experiments above: 2-6 Workers, 28-84 GB Memory, Standard_DS3_v2(worker)/Standard_DS4_v2(driver)

#### Grid Search Performed
| Model | Hyperparameters Explored                                          | Features | Best Model Runtime (min) |
|------------|-----------------------------------------------------------|----------|---------------|
| MLP   | 1_layer neutrons[10,30,50], 2_layer neutrons[5,4;20;10] | 8      | 37           |
| XGBoost    | max_depth: [3, 5, 8, 10], eta: [0.1, 0.15, 0.2], num_round: [50, 100] | 10       | 60            |
| Random Forest    | numTrees: [3, 10, 25], maxDepth: [3, 5] | 13       | 58            |
#### Interpretation of Results
The XGBoost model was tested with various hyperparameters and completed in 60 minutes using 10 features. During the cross-validation process, Fold 2 had the highest F-beta score of 0.62, indicating a balanced precision of 0.42 and recall of 0.70. The other folds showed consistent performance, with recall generally higher than precision. On the test set, the F-beta score was 0.56, with precision at 0.40 and recall at 0.63, aligning well with the cross-validation results. The model's performance across different folds and the test set was relatively consistent, suggesting reliable and robust model performance. The test set results confirm that the model consistently balances precision and recall, with slight variations across different data splits.

The Random Forest model was tested with various hyperparameters in a grid search and completed in 58 minutes across all parameter combinations and folds. Its relatively quick runtime per trained model can be attributed to its bagging architecture and Spark's ability to train individual decision trees in parallel. Similar to the XGBoost model, the Random Forest model perfomed the best on Fold 2 with an F-beta score of 0.61 and a precision of 0.34 and recall of 0.75, and it performed consistently across the folds. Out of all the models, it performed the best on the test set. There can be several reasons for this. The model has a high recall because it is more inclined to predict the positive class in general due to the data rebalancing. It has a low precision due to the relatively shallow depth of the trees along with the data rebalancing. The splits the trees generate may not be completely pure, so it gets a lot of the negative class as positive. However, with the inherent feature selection capability of decision trees, the features that were used happen to capture much of the positive class.

When comparing these results to the baseline logistic regression model, which completed in 16 minutes with an F-beta score of 0.52 on the test set, it is clear that while logistic regression offers a simple and computationally efficient solution, it falls short in balancing precision and recall compared to the more complex models like XGBoost and Random Forest. The Multilayer Perceptron (MLP) models, which took 37 and 39 minutes respectively, did not offer significant improvements over logistic regression, with the two-layer MLP model even showing signs of overfitting. This suggests that while MLPs are more flexible than logistic regression, they may not capture the underlying data patterns as effectively as tree-based models in this context.

The final model is an ensemble model, which combined the positive class probability from XGBoost and RF, and have a logistic regression as the output layer, achieved performance with an F-2 score of 0.55 on the test set, precision of 0.37, and recall of 0.62. Although its recall and F-beta score were slightly lower than those of the Random Forest model, the ensemble model offered a better balance between precision and recall across all folds and test set, demonstrating strong generalization capabilities. This balance, combined with its consistent performance across different data splits and its ability to mitigate the weaknesses of individual models, makes the ensemble model a compelling choice for maximizing overall predictive accuracy and ensuring reliable outcomes in varied deployment scenarios.



## X. Conclusion

This project focused on developing machine learning models to predict U.S. flight delays at least two hours in advance, a critical task for improving airline operational efficiency and passenger satisfaction. Accurate predictions of flight delays can enable better resource management, reduce operational costs, and enhance the overall travel experience. Given the complexity and impact of flight delays, creating reliable prediction models using a vast dataset was essential.

We hypothesized that previous delays, weather, or large hubs contribute to current flight delays. Through rigorous experimentation, we employed feature selection methods like Chi-squared tests for categorical features and ANOVA F-tests for continuous features. And we choose F-2 as our main metric, which focusing on recall. Our experiments includes logistic regression, neural network, XGboost, random forest and ensemble models. These techniques allowed us to identify the most influential features, the best hyperparameters, and if the model is overfitting.

Our key findings demonstrated that the Random Forest model, which incorporated a well-selected feature set, achieved the highest F-2 score of 0.57 and highest recall of 0.72 on the test set. XGBoost, while also highly effective, achieved an F-2 score of 0.56 and demonstrated robust performance, particularly in cross-validation. The ensemble model, which combined XGBoost and Random Forest with logistic regression, also performed admirably, with an F-2 score of 0.55 on test set, providing a more balanced approach across various metrics.

The significance of our results lies in their potential application within the airline industry. By leveraging these models, airlines can better anticipate delays, optimize scheduling, and improve the passenger experience. The consistent performance of these models across different data splits suggests their robustness and reliability in real-world applications.

Looking ahead, the project has several promising avenues for future development. Enhancing the models with additional real-time data sources, such as live air traffic control information or real-time weather updates, could further improve model performance. Additionally, exploring more deep learning techniques or more sophisticated ensembling methods might yield even better results. Finally, deploying these models in a live environment and continuously refining them based on new data would be the next logical step to fully realize their potential in reducing flight delays.
